# Attentional Networks in Computer Vision
Prepared by Comp411 Teaching Unit (TA Can Küçüksözen) in the context of Computer Vision with Deep Learning Course. Do not hesitate to ask in case you have any questions, contact me at: ckucuksozen19@ku.edu.tr

Up until this point, we have worked with deep fully-connected networks, convolutional networks and recurrent networks using them to explore different optimization strategies and network architectures. Fully-connected networks are a good testbed for experimentation because they are very computationally efficient, on the other hand, most successful image processing methods use convolutional networks. However recent state-of-the-art results on computer vision realm are acquired using Attentional layers and Transformer architectures.

First you will implement several layer types that are used in fully attentional networks. You will then use these layers to train an Attentional Image Classification network, specifically a smaller version of Vision Transformer (VIT) on the CIFAR-10 dataset. The original paper can be accessed via the following link: https://arxiv.org/pdf/2010.11929.pdf

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as T

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./comp411/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./comp411/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./comp411/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


# Part II. Barebones Transformers: Self-Attentional Layer

Here you will complete the implementation of the Pytorch nn.module `SelfAttention`, which will perform the forward pass of a self-attentional layer. Our implementation of the SelfAttentional layer will include three distinct fully connected layers which will be responsible of:

1. A fully connected layer, `W_Q`, which will be used to project our input into `queries`
2. A fully connected layer, `W_K`, which will be used to project our input into `keys`
3. A fully connected layer, `W_V`, which will be used to project our input into `values`

After defining such three fully connected layers, and obtain our `queries, keys, and values` variables at the beginning of our forward pass, the following operations should be carried out in order to complete the attentional layer implementation.

1. Seperate each of `query, key, and value` projections into their respective heads. In other words, split the feature vector dimension of each matrix into necessarry number of chunks.

2. Compute the `Attention Scores` between each pair of sequence elements via conducting a scaled dot product operation between every pair of `queries` and `keys`. Note that `Attention Scores` matrix should have the size of `[# of queries , # of keys]`

3. Calculate the `Attention Weights` of each query by applying the non-linear `Softmax` normalization accross the `keys` dimension of the `Attention Scores` matrix.

4. Obtain the output combination of `values` by matrix multiplying `Attention Weights` with `values`

5. Reassemble heads into one flat vector and return the output.

**HINT**: For a more detailed explanation of the self attentional layer, examine the Appendix A of the original ViT manuscript here:  https://arxiv.org/pdf/2010.11929.pdf 

In [4]:
class SelfAttention(nn.Module):
    
    def __init__(self, input_dims, head_dims=128, num_heads=2,  bias=False):
        super(SelfAttention, self).__init__()
        
        ## initialize module's instance variables
        self.input_dims = input_dims
        self.head_dims = head_dims
        self.num_heads = num_heads
        self.proj_dims = head_dims * num_heads
        
        ## Declare module's parameters
        self.W_Q = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_K = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_V = nn.Linear(input_dims, self.proj_dims,bias=bias)
        
        self.W_O = nn.Linear(self.proj_dims,self.proj_dims,bias=bias)

        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)

    def forward(self, x):
        ## Input of shape, [B, N, D] where:
        ## - B denotes the batch size
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D corresponds to model dimensionality
        b,n,d = x.shape
        
        ## Construct queries,keys,values
        q_ = self.W_Q(x)
        k_ = self.W_K(x)
        v_ = self.W_V(x)
        
        ## Seperate q,k,v into their corresponding heads,
        ## After this operation each q,k,v will have the shape: [B,H,N,D//H] where
        ## - B denotes the batch size
        ## - H denotes number of heads
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D//H corresponds to per head dimensionality
        q, k, v = map(lambda z: torch.reshape(z, (b,n,self.num_heads,self.head_dims)).permute(0,2,1,3), [q_,k_,v_])
       
        attn_out = None
        
        #########################################################################################
        # TODO: Complete the forward pass of the SelfAttention layer, follow the comments below #
        #########################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        ## Compute attention logits. Note that this operation is implemented as a
        ## batched matrix multiplication between q and k, the output is scaled by 1/(D//H)^(1/2)
        ## inputs are queries and keys that are both of size [B,H,N,D//H]
        ## Output Attention logits should have the size: [B,H,N,N]
        
        intermdeiary = torch.matmul(q,k.transpose(-2,-1)) * (1/((d//self.num_heads)**(1/2)))
    
        ## Compute attention Weights. Recall that this operation is conducted as a
        ## Softmax Normalization across the keys dimension. 
        ## Hint: You can apply the Softmax operation across the final dimension
        
        # attn_weights = nn.softmax(intermediary,dim=-1)
        attn_weights = intermdeiary.softmax(dim=-1)
    
        ## Compute attention output values. Bear in mind that this operation is applied as a 
        ## batched matrix multiplication between the Attention Weights matrix and 
        ## the values tensor. After computing output values, the output should be reshaped
        ## Inputs are Attention Weights with size [B, H, N, N], values with size [B, H, N, D//H]
        ## Output should be of size [B, N, D]
        ## Hint: you should use torch.matmul, torch.permute, torch.reshape in that order
        ##       (or any other equivalent torch operations)
        
        scores = torch.matmul(attn_weights,v)
        
        out = scores.reshape(b,n,self.proj_dims)
        
        ## Compute output feature map. This operation is just passing the concatenated attention 
        ## output that we have just obtained through a final projection layer W_O.
        ## Both the input and the output should be of size [B, N, D]
        
        attn_out = self.W_O(out)        
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             
        ################################################################################
        
        return attn_out

After defining the forward pass of the Self-Attentional Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [5]:
def test_self_attn_layer():
    x = torch.zeros((64, 16, 32), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 32
    layer = SelfAttention(32,64,4)
    out = layer(x)
    print(out.size())  # you should see [64,16,256]
test_self_attn_layer()

torch.Size([64, 16, 256])


# Part III. Barebones Transformers: Transformer Encoder Block

Here you will complete the implementation of the Pytorch nn.module `TransformerBlock`, which will perform the forward pass of a Transfomer Encoder Block. You can refer to Figure 1 of the original manuscript of ViT from this link: https://arxiv.org/pdf/2010.11929.pdf in order to get yourself familiar with the architecture.



In [6]:
## Implementation of a two layer GELU activated Fully Connected Network is provided for you below:

class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, bias=True):
        super().__init__()
        
        self.fc_1 = nn.Linear(input_dims, hidden_dims, bias=bias)
        self.fc_2 = nn.Linear(hidden_dims, output_dims, bias=bias)
        
        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)
        
    def forward(self, x):
        o = F.gelu(self.fc_1(x))
        o = self.fc_2(o)
        return o

In [8]:
## Build from scratch a TransformerBlock Module. Note that the architecture of this
## module follows a simple computational pipeline:
## input --> layernorm --> SelfAttention --> skip connection 
##       --> layernorm --> MLP ---> skip connection ---> output
## Note that the TransformerBlock module works on a single hidden dimension hidden_dims,
## in order to faciliate skip connections with ease. Be careful about the input arguments
## to the SelfAttention block.


class TransformerBlock(nn.Module):
    def __init__(self, hidden_dims, num_heads=4, bias=False):
        super(TransformerBlock, self).__init__()
        
###############################################################
# TODO: Complete the consturctor of  TransformerBlock module  #
###############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****

        self.layerNorm_1 = nn.LayerNorm(hidden_dims)
        self.selfAttention = SelfAttention(hidden_dims, hidden_dims//num_heads, num_heads, bias=bias)
        self.layerNorm_2 = nn.LayerNorm(hidden_dims)
        self.fullyConnected = MLP(hidden_dims,hidden_dims,hidden_dims,bias=bias)
        

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #             
###################################################################
        
    def forward(self, x):
        
##############################################################
# TODO: Complete the forward of TransformerBlock module      #
##############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****

        normalized_1 = self.layerNorm_1(x)
        attn_out = self.selfAttention(normalized_1)
        skipped_1 = attn_out + x
        normalized_2 = self.layerNorm_2(skipped_1)
        fullyConnected_out = self.fullyConnected(normalized_2)
        skipped_2 = fullyConnected_out + x
        
        return skipped_2
        
 # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #             
###################################################################

After defining the forward pass of the Transformer Block Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [9]:
def test_transfomerblock_layer():
    x = torch.zeros((64, 16, 128), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 128
    layer = TransformerBlock(128,4) # hidden dims size 128, heads size 4
    out = layer(x)
    print(out.size())  # you should see [64,16,128]
test_transfomerblock_layer()

torch.Size([64, 16, 128])


# Part IV The Vision Transformer (ViT)

The final implementation for the Pytorch nn.module `ViT` is given to you below, which will perform the forward pass of the Vision Transformer. Study it and get yourself familiar with the API.


In [10]:
class ViT(nn.Module):
    def __init__(self, hidden_dims, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4, bias=False):
        super(ViT, self).__init__()
                
        ## initialize module's instance variables
        self.hidden_dims = hidden_dims
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.num_trans_layers = num_trans_layers
        self.num_heads = num_heads
        self.image_k = image_k
        self.patch_k = patch_k
        
        self.image_height = self.image_width = image_k
        self.patch_height = self.patch_width = patch_k
        
        assert self.image_height % self.patch_height == 0 and self.image_width % self.patch_width == 0,\
                'Image size must be divisible by the patch size.'

        self.num_patches = (self.image_height // self.patch_height) * (self.image_width // self.patch_width)
        self.patch_flat_len = self.patch_height * self.patch_width
        
        ## Declare module's parameters
        
        ## ViT's flattened patch embedding projection:
        self.linear_embed = nn.Linear(self.input_dims*self.patch_flat_len, self.hidden_dims)
        
        ## Learnable positional embeddings, an embedding is learned for each patch location and the class token
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, self.hidden_dims))
        
        ## Learnable classt token and its index among attention sequence elements.
        self.cls_token = nn.Parameter(torch.randn(1,1,self.hidden_dims))
        self.cls_index = torch.LongTensor([0])
        
        ## Declare cascaded Transformer blocks:
        transformer_encoder_list = []
        for _ in range(self.num_trans_layers):
            transformer_encoder_list.append(TransformerBlock(self.hidden_dims, self.num_heads, bias))
        self.transformer_encoder = nn.Sequential(*transformer_encoder_list)
        
        ## Declare the output mlp:
        self.out_mlp = MLP(self.hidden_dims, self.hidden_dims, self.output_dims)
         
    def unfold(self, x, f = 7, st = 4, p = 0):
        ## Create sliding window pathes using nn.Functional.unfold
        ## Input dimensions: [B,D,H,W] where
        ## --B : input batch size
        ## --D : input channels
        ## --H, W: input height and width
        ## Output dimensions: [B,N,H*W,D]
        ## --N : number of patches, decided according to sliding window kernel size (f),
        ##      sliding window stride and padding.
        b,d,h,w = x.shape
        x_unf = F.unfold(x, (f,f), stride=st, padding=p)    
        x_unf = torch.reshape(x_unf.permute(0,2,1), (b,-1,d,f*f)).transpose(-1,-2)
        n = x_unf.size(1)
        return x_unf,n
    
    def forward(self, x):
        b = x.size(0)
        ## create sliding window patches from the input image
        x_patches,n = self.unfold(x, self.patch_height, self.patch_height, 0)
        ## flatten each patch into a 1d vector: i.e. 3x4x4 image patch turned into 1x1x48
        x_patch_flat = torch.reshape(x_patches, (b,n,-1))
        ## linearly embed each flattened patch
        x_embed = self.linear_embed(x_patch_flat)
        
        ## retrieve class token 
        cls_tokens = self.cls_token.repeat(b,1,1)
        ## concatanate class token to input patches
        xcls_embed = torch.cat([cls_tokens, x_embed], dim=-2)
        
        ## add positional embedding to input patches + class token 
        xcls_pos_embed = xcls_embed + self.pos_embedding
        
        ## pass through the transformer encoder
        trans_out = self.transformer_encoder(xcls_pos_embed)
        
        ## select the class token 
        out_cls_token = torch.index_select(trans_out, -2, self.cls_index.to(trans_out.device))
        
        ## create output
        out = self.out_mlp(out_cls_token)
        
        return out.squeeze(-2)

After defining the forward pass of the ViT above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [11]:
def test_vit():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size 3,32,32
    model = ViT(hidden_dims=128, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4)
    out = model(x)
    print(out.size())  # you should see [64,10]
test_vit()

torch.Size([64, 10])


# Part V. Train the ViT

### Check Accuracy
Given any minibatch of input data and desired targets, we can check the classification accuracy of a neural network. 

The check_batch_accuracy function is provided for you below:

In [12]:
def check_batch_accuracy(out, target,eps=1e-7):
    b, c = out.shape
    with torch.no_grad():
        _, pred = out.max(-1) 
        correct = np.sum(np.equal(pred.cpu().numpy(), target.cpu().numpy()))
    return correct, np.float(correct) / (b)

### Training Loop
As we have already seen in the Second Assignment, in our PyTorch based training loops, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [13]:
def train(network, optimizer, trainloader):
    """
    Train a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - trainloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall training accuracy for the epoch
    """
    print('\nEpoch: %d' % epoch)
    network.train()  # put model to training mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = Variable(inputs.to(device)), targets.to(device)  # move to device, e.g. GPU
            
        outputs = network(inputs)
        loss =  F.cross_entropy(outputs, targets)
            
        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad() 

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()
            
        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()
            
        loss = loss.detach()
        train_loss += loss.item()
        correct_p, _ = check_batch_accuracy(outputs, targets) 
        correct += correct_p
        total += targets.size(0)

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    return 100.*correct/total

### Evaluation Loop
We have also prepared a Evaluation loop in order to determine our networks capabilities in terms of classification accuracy on a given dataset, either the training, or the validation split

In [14]:
def evaluate(network, evalloader):
    """
    Evaluate a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - evalloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall evaluation accuracy for the epoch
    """
    network.eval() # put model to evaluation mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    eval_loss = 0
    correct = 0
    total = 0
    print('\n---- Evaluation in process ----')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(evalloader):
            inputs, targets = inputs.to(device), targets.to(device) # move to device, e.g. GPU
            outputs = network(inputs)
            loss = F.cross_entropy(outputs, targets)
            
            eval_loss += loss.item()
            correct_p, _ = check_batch_accuracy(outputs, targets)
            correct += correct_p
            total += targets.size(0)
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (eval_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return 100.*correct/total

### Overfit a ViT
Now we are ready to run the training loop. A nice trick is to train your model with just a few training samples in order to see if your implementation is actually bug free. 

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `ViT`. 

You also need to define an optimizer that tracks all the learnable parameters inside `ViT`. We prefer to use `Adam` optimizer for this part.

You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

In [15]:
sample_idx_tr = torch.randperm(len(cifar10_train))[:100]
sample_idx_val = torch.randperm(len(cifar10_train))[-100:]

trainset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_tr)
valset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_val)

print("For overfitting experiments, the subset of the dataset that is used has {} sample images".format(len(trainset_sub)))

batch_size_sub = 25
trainloader_sub = torch.utils.data.DataLoader(trainset_sub, batch_size=batch_size_sub, shuffle=True)
valloader_sub = torch.utils.data.DataLoader(valset_sub, batch_size=batch_size_sub, shuffle=False)

print('==> Data ready, batchsize = {}'.format(batch_size_sub))

For overfitting experiments, the subset of the dataset that is used has 100 sample images
==> Data ready, batchsize = 25


In [16]:
learning_rate = 0.001
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

model = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = ViT(hidden_dims, input_dims, output_dims, num_trans_layers, num_heads, image_k, patch_k)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
eval_accs=[]
for epoch in range(15):
    tr_acc = train(model, optimizer, trainloader_sub)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    eval_acc = evaluate(model, valloader_sub)
    print('Evaluation of Epoch {} is completed, Validation accuracy for this epoch is {}'\
              .format(epoch, eval_acc))  
    tr_accs.append(tr_acc)
    eval_accs.append(eval_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final val set accuracy is {}".format(eval_accs[-1]))


Epoch: 0
Loss: 3.199 | Acc: 4.000% (1/25)
Loss: 3.611 | Acc: 8.000% (4/50)
Loss: 3.512 | Acc: 6.667% (5/75)
Loss: 3.601 | Acc: 6.000% (6/100)
Epoch 0 of training is completed, Training accuracy for this epoch is 6.0

---- Evaluation in process ----
Loss: 3.147 | Acc: 4.000% (1/25)
Loss: 2.938 | Acc: 6.000% (3/50)
Loss: 2.956 | Acc: 5.333% (4/75)
Loss: 3.060 | Acc: 6.000% (6/100)
Evaluation of Epoch 0 is completed, Validation accuracy for this epoch is 6.0

Epoch: 1
Loss: 2.659 | Acc: 8.000% (2/25)
Loss: 2.564 | Acc: 12.000% (6/50)
Loss: 2.683 | Acc: 9.333% (7/75)
Loss: 2.564 | Acc: 10.000% (10/100)
Epoch 1 of training is completed, Training accuracy for this epoch is 10.0

---- Evaluation in process ----
Loss: 2.796 | Acc: 8.000% (2/25)
Loss: 2.596 | Acc: 14.000% (7/50)
Loss: 2.539 | Acc: 14.667% (11/75)
Loss: 2.508 | Acc: 18.000% (18/100)
Evaluation of Epoch 1 is completed, Validation accuracy for this epoch is 18.0

Epoch: 2
Loss: 2.277 | Acc: 20.000% (5/25)
Loss: 2.337 | Acc: 20.00

## Train the net
By training the four-layer ViT network for three epochs, with untuned hyperparameters that are initialized as below,  you should achieve greater than 50% accuracy both on the training set and the test set:

In [18]:
learning_rate = 0.001
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims, input_dims, output_dims, num_trans_layers, num_heads, image_k, patch_k)
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
test_accs=[]
for epoch in range(3):
    tr_acc = train(network, optimizer, loader_train)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    test_acc = evaluate(network, loader_test)
    print('Evaluation of Epoch {} is completed, Test accuracy for this epoch is {}'\
              .format(epoch, test_acc))  
    
    tr_accs.append(tr_acc)
    test_accs.append(test_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final test set accuracy is {}".format(test_accs[-1]))


Epoch: 0
Loss: 4.245 | Acc: 6.250% (4/64)
Loss: 3.621 | Acc: 8.594% (11/128)
Loss: 3.599 | Acc: 7.812% (15/192)
Loss: 3.415 | Acc: 9.766% (25/256)
Loss: 3.330 | Acc: 10.625% (34/320)
Loss: 3.258 | Acc: 10.417% (40/384)
Loss: 3.155 | Acc: 10.938% (49/448)
Loss: 3.053 | Acc: 12.891% (66/512)
Loss: 2.980 | Acc: 13.715% (79/576)
Loss: 2.922 | Acc: 14.062% (90/640)
Loss: 2.877 | Acc: 13.636% (96/704)
Loss: 2.821 | Acc: 13.932% (107/768)
Loss: 2.768 | Acc: 14.062% (117/832)
Loss: 2.746 | Acc: 13.839% (124/896)
Loss: 2.716 | Acc: 13.646% (131/960)
Loss: 2.681 | Acc: 13.965% (143/1024)
Loss: 2.646 | Acc: 14.154% (154/1088)
Loss: 2.617 | Acc: 14.410% (166/1152)
Loss: 2.588 | Acc: 14.885% (181/1216)
Loss: 2.571 | Acc: 14.844% (190/1280)
Loss: 2.547 | Acc: 15.030% (202/1344)
Loss: 2.541 | Acc: 15.128% (213/1408)
Loss: 2.529 | Acc: 15.285% (225/1472)
Loss: 2.516 | Acc: 15.560% (239/1536)
Loss: 2.502 | Acc: 15.812% (253/1600)
Loss: 2.486 | Acc: 15.745% (262/1664)
Loss: 2.472 | Acc: 15.914% (275/17

Loss: 1.896 | Acc: 30.886% (4329/14016)
Loss: 1.895 | Acc: 30.966% (4360/14080)
Loss: 1.894 | Acc: 31.003% (4385/14144)
Loss: 1.893 | Acc: 31.053% (4412/14208)
Loss: 1.892 | Acc: 31.075% (4435/14272)
Loss: 1.891 | Acc: 31.124% (4462/14336)
Loss: 1.890 | Acc: 31.160% (4487/14400)
Loss: 1.889 | Acc: 31.188% (4511/14464)
Loss: 1.889 | Acc: 31.209% (4534/14528)
Loss: 1.888 | Acc: 31.223% (4556/14592)
Loss: 1.887 | Acc: 31.277% (4584/14656)
Loss: 1.887 | Acc: 31.298% (4607/14720)
Loss: 1.885 | Acc: 31.399% (4642/14784)
Loss: 1.884 | Acc: 31.466% (4672/14848)
Loss: 1.883 | Acc: 31.532% (4702/14912)
Loss: 1.882 | Acc: 31.557% (4726/14976)
Loss: 1.881 | Acc: 31.543% (4744/15040)
Loss: 1.880 | Acc: 31.574% (4769/15104)
Loss: 1.879 | Acc: 31.606% (4794/15168)
Loss: 1.879 | Acc: 31.611% (4815/15232)
Loss: 1.878 | Acc: 31.616% (4836/15296)
Loss: 1.876 | Acc: 31.686% (4867/15360)
Loss: 1.875 | Acc: 31.723% (4893/15424)
Loss: 1.874 | Acc: 31.754% (4918/15488)
Loss: 1.872 | Acc: 31.790% (4944/15552)


Loss: 1.750 | Acc: 36.419% (9906/27200)
Loss: 1.750 | Acc: 36.447% (9937/27264)
Loss: 1.749 | Acc: 36.472% (9967/27328)
Loss: 1.749 | Acc: 36.492% (9996/27392)
Loss: 1.749 | Acc: 36.517% (10026/27456)
Loss: 1.747 | Acc: 36.552% (10059/27520)
Loss: 1.747 | Acc: 36.561% (10085/27584)
Loss: 1.747 | Acc: 36.574% (10112/27648)
Loss: 1.746 | Acc: 36.580% (10137/27712)
Loss: 1.745 | Acc: 36.611% (10169/27776)
Loss: 1.745 | Acc: 36.627% (10197/27840)
Loss: 1.745 | Acc: 36.622% (10219/27904)
Loss: 1.745 | Acc: 36.624% (10243/27968)
Loss: 1.744 | Acc: 36.644% (10272/28032)
Loss: 1.744 | Acc: 36.671% (10303/28096)
Loss: 1.743 | Acc: 36.694% (10333/28160)
Loss: 1.743 | Acc: 36.678% (10352/28224)
Loss: 1.742 | Acc: 36.694% (10380/28288)
Loss: 1.742 | Acc: 36.696% (10404/28352)
Loss: 1.742 | Acc: 36.701% (10429/28416)
Loss: 1.742 | Acc: 36.706% (10454/28480)
Loss: 1.742 | Acc: 36.705% (10477/28544)
Loss: 1.742 | Acc: 36.682% (10494/28608)
Loss: 1.741 | Acc: 36.698% (10522/28672)
Loss: 1.741 | Acc: 3

Loss: 1.675 | Acc: 39.259% (15804/40256)
Loss: 1.674 | Acc: 39.281% (15838/40320)
Loss: 1.674 | Acc: 39.263% (15856/40384)
Loss: 1.674 | Acc: 39.265% (15882/40448)
Loss: 1.674 | Acc: 39.280% (15913/40512)
Loss: 1.674 | Acc: 39.282% (15939/40576)
Loss: 1.674 | Acc: 39.289% (15967/40640)
Loss: 1.674 | Acc: 39.291% (15993/40704)
Loss: 1.674 | Acc: 39.288% (16017/40768)
Loss: 1.673 | Acc: 39.300% (16047/40832)
Loss: 1.673 | Acc: 39.329% (16084/40896)
Loss: 1.672 | Acc: 39.348% (16117/40960)
Loss: 1.671 | Acc: 39.377% (16154/41024)
Loss: 1.671 | Acc: 39.389% (16184/41088)
Loss: 1.671 | Acc: 39.403% (16215/41152)
Loss: 1.671 | Acc: 39.402% (16240/41216)
Loss: 1.670 | Acc: 39.421% (16273/41280)
Loss: 1.670 | Acc: 39.418% (16297/41344)
Loss: 1.670 | Acc: 39.434% (16329/41408)
Loss: 1.669 | Acc: 39.456% (16363/41472)
Loss: 1.669 | Acc: 39.474% (16396/41536)
Loss: 1.668 | Acc: 39.481% (16424/41600)
Loss: 1.668 | Acc: 39.485% (16451/41664)
Loss: 1.668 | Acc: 39.491% (16479/41728)
Loss: 1.667 | Ac

Loss: 1.467 | Acc: 47.466% (2248/4736)
Loss: 1.467 | Acc: 47.542% (2282/4800)
Loss: 1.464 | Acc: 47.615% (2316/4864)
Loss: 1.464 | Acc: 47.687% (2350/4928)
Loss: 1.464 | Acc: 47.676% (2380/4992)
Loss: 1.462 | Acc: 47.666% (2410/5056)
Loss: 1.463 | Acc: 47.617% (2438/5120)
Loss: 1.462 | Acc: 47.589% (2467/5184)
Loss: 1.464 | Acc: 47.542% (2495/5248)
Loss: 1.465 | Acc: 47.459% (2521/5312)
Loss: 1.469 | Acc: 47.340% (2545/5376)
Loss: 1.469 | Acc: 47.279% (2572/5440)
Loss: 1.469 | Acc: 47.347% (2606/5504)
Loss: 1.469 | Acc: 47.306% (2634/5568)
Loss: 1.469 | Acc: 47.390% (2669/5632)
Loss: 1.470 | Acc: 47.419% (2701/5696)
Loss: 1.467 | Acc: 47.465% (2734/5760)
Loss: 1.466 | Acc: 47.527% (2768/5824)
Loss: 1.468 | Acc: 47.418% (2792/5888)
Loss: 1.469 | Acc: 47.379% (2820/5952)
Loss: 1.470 | Acc: 47.291% (2845/6016)
Loss: 1.471 | Acc: 47.220% (2871/6080)
Loss: 1.470 | Acc: 47.282% (2905/6144)
Loss: 1.471 | Acc: 47.197% (2930/6208)
Loss: 1.473 | Acc: 47.066% (2952/6272)
Loss: 1.473 | Acc: 47.143

...

Loss: 1.240 | Acc: 56.530% (1411/2496)
Loss: 1.240 | Acc: 56.445% (1445/2560)
Loss: 1.240 | Acc: 56.441% (1481/2624)
Loss: 1.238 | Acc: 56.473% (1518/2688)
Loss: 1.239 | Acc: 56.395% (1552/2752)
Loss: 1.239 | Acc: 56.357% (1587/2816)
Loss: 1.240 | Acc: 56.285% (1621/2880)
Loss: 1.238 | Acc: 56.284% (1657/2944)
Loss: 1.241 | Acc: 56.084% (1687/3008)
Loss: 1.241 | Acc: 56.087% (1723/3072)
Loss: 1.241 | Acc: 55.995% (1756/3136)
Loss: 1.237 | Acc: 56.188% (1798/3200)
Loss: 1.238 | Acc: 56.189% (1834/3264)
Loss: 1.240 | Acc: 56.070% (1866/3328)
Loss: 1.238 | Acc: 56.191% (1906/3392)
Loss: 1.237 | Acc: 56.076% (1938/3456)
Loss: 1.241 | Acc: 55.852% (1966/3520)
Loss: 1.240 | Acc: 55.831% (2001/3584)
Loss: 1.244 | Acc: 55.592% (2028/3648)
Loss: 1.242 | Acc: 55.738% (2069/3712)
Loss: 1.238 | Acc: 55.932% (2112/3776)
Loss: 1.240 | Acc: 55.911% (2147/3840)
Loss: 1.242 | Acc: 55.738% (2176/3904)
Loss: 1.240 | Acc: 55.771% (2213/3968)
Loss: 1.242 | Acc: 55.754% (2248/4032)
Loss: 1.244 | Acc: 55.640

Loss: 1.258 | Acc: 54.756% (8831/16128)
Loss: 1.257 | Acc: 54.768% (8868/16192)
Loss: 1.257 | Acc: 54.792% (8907/16256)
Loss: 1.257 | Acc: 54.767% (8938/16320)
Loss: 1.257 | Acc: 54.773% (8974/16384)
Loss: 1.257 | Acc: 54.760% (9007/16448)
Loss: 1.257 | Acc: 54.742% (9039/16512)
Loss: 1.258 | Acc: 54.748% (9075/16576)
Loss: 1.258 | Acc: 54.718% (9105/16640)
Loss: 1.258 | Acc: 54.717% (9140/16704)
Loss: 1.258 | Acc: 54.735% (9178/16768)
Loss: 1.258 | Acc: 54.735% (9213/16832)
Loss: 1.258 | Acc: 54.717% (9245/16896)
Loss: 1.259 | Acc: 54.711% (9279/16960)
Loss: 1.259 | Acc: 54.699% (9312/17024)
Loss: 1.259 | Acc: 54.688% (9345/17088)
Loss: 1.259 | Acc: 54.705% (9383/17152)
Loss: 1.259 | Acc: 54.699% (9417/17216)
Loss: 1.259 | Acc: 54.699% (9452/17280)
Loss: 1.260 | Acc: 54.653% (9479/17344)
Loss: 1.259 | Acc: 54.665% (9516/17408)
Loss: 1.259 | Acc: 54.665% (9551/17472)
Loss: 1.259 | Acc: 54.688% (9590/17536)
Loss: 1.258 | Acc: 54.693% (9626/17600)
Loss: 1.259 | Acc: 54.671% (9657/17664)


Loss: 1.252 | Acc: 54.797% (15957/29120)
Loss: 1.252 | Acc: 54.807% (15995/29184)
Loss: 1.252 | Acc: 54.797% (16027/29248)
Loss: 1.252 | Acc: 54.807% (16065/29312)
Loss: 1.253 | Acc: 54.783% (16093/29376)
Loss: 1.253 | Acc: 54.779% (16127/29440)
Loss: 1.253 | Acc: 54.769% (16159/29504)
Loss: 1.253 | Acc: 54.772% (16195/29568)
Loss: 1.254 | Acc: 54.772% (16230/29632)
Loss: 1.253 | Acc: 54.785% (16269/29696)
Loss: 1.253 | Acc: 54.775% (16301/29760)
Loss: 1.252 | Acc: 54.802% (16344/29824)
Loss: 1.252 | Acc: 54.831% (16388/29888)
Loss: 1.252 | Acc: 54.831% (16423/29952)
Loss: 1.252 | Acc: 54.831% (16458/30016)
Loss: 1.252 | Acc: 54.824% (16491/30080)
Loss: 1.252 | Acc: 54.814% (16523/30144)
Loss: 1.252 | Acc: 54.820% (16560/30208)
Loss: 1.252 | Acc: 54.803% (16590/30272)
Loss: 1.252 | Acc: 54.800% (16624/30336)
Loss: 1.252 | Acc: 54.812% (16663/30400)
Loss: 1.252 | Acc: 54.806% (16696/30464)
Loss: 1.252 | Acc: 54.815% (16734/30528)
Loss: 1.252 | Acc: 54.808% (16767/30592)
Loss: 1.253 | Ac

Loss: 1.255 | Acc: 54.776% (22927/41856)
Loss: 1.255 | Acc: 54.769% (22959/41920)
Loss: 1.255 | Acc: 54.773% (22996/41984)
Loss: 1.255 | Acc: 54.778% (23033/42048)
Loss: 1.255 | Acc: 54.785% (23071/42112)
Loss: 1.255 | Acc: 54.775% (23102/42176)
Loss: 1.255 | Acc: 54.773% (23136/42240)
Loss: 1.255 | Acc: 54.768% (23169/42304)
Loss: 1.255 | Acc: 54.761% (23201/42368)
Loss: 1.255 | Acc: 54.775% (23242/42432)
Loss: 1.255 | Acc: 54.775% (23277/42496)
Loss: 1.255 | Acc: 54.760% (23306/42560)
Loss: 1.255 | Acc: 54.770% (23345/42624)
Loss: 1.255 | Acc: 54.767% (23379/42688)
Loss: 1.255 | Acc: 54.760% (23411/42752)
Loss: 1.255 | Acc: 54.760% (23446/42816)
Loss: 1.255 | Acc: 54.762% (23482/42880)
Loss: 1.255 | Acc: 54.774% (23522/42944)
Loss: 1.255 | Acc: 54.783% (23561/43008)
Loss: 1.255 | Acc: 54.794% (23601/43072)
Loss: 1.255 | Acc: 54.787% (23633/43136)
Loss: 1.255 | Acc: 54.792% (23670/43200)
Loss: 1.255 | Acc: 54.794% (23706/43264)
Loss: 1.255 | Acc: 54.782% (23736/43328)
Loss: 1.255 | Ac

Loss: 1.274 | Acc: 54.572% (3318/6080)
Loss: 1.273 | Acc: 54.541% (3351/6144)
Loss: 1.275 | Acc: 54.494% (3383/6208)
Loss: 1.276 | Acc: 54.448% (3415/6272)
Loss: 1.274 | Acc: 54.482% (3452/6336)
Loss: 1.275 | Acc: 54.375% (3480/6400)
Loss: 1.277 | Acc: 54.270% (3508/6464)
Loss: 1.276 | Acc: 54.320% (3546/6528)
Loss: 1.277 | Acc: 54.308% (3580/6592)
Loss: 1.276 | Acc: 54.312% (3615/6656)
Loss: 1.277 | Acc: 54.241% (3645/6720)
Loss: 1.277 | Acc: 54.260% (3681/6784)
Loss: 1.275 | Acc: 54.322% (3720/6848)
Loss: 1.278 | Acc: 54.225% (3748/6912)
Loss: 1.280 | Acc: 54.114% (3775/6976)
Loss: 1.282 | Acc: 54.048% (3805/7040)
Loss: 1.280 | Acc: 54.110% (3844/7104)
Loss: 1.282 | Acc: 54.032% (3873/7168)
Loss: 1.284 | Acc: 53.982% (3904/7232)
Loss: 1.283 | Acc: 53.975% (3938/7296)
Loss: 1.281 | Acc: 54.062% (3979/7360)
Loss: 1.282 | Acc: 53.987% (4008/7424)
Loss: 1.281 | Acc: 53.953% (4040/7488)
Loss: 1.282 | Acc: 53.906% (4071/7552)
Loss: 1.284 | Acc: 53.782% (4096/7616)
Loss: 1.283 | Acc: 53.841